In [1]:
import pandas as pd
import warnings
import re
warnings.filterwarnings("ignore")
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [2]:
model = SentenceTransformer('multi-qa-mpnet-base-cos-v1')
brand=pd.read_csv("Data/brand_category_clean.csv")
brand["BRAND"]=brand["BRAND"].apply(str)
cat=pd.read_csv("Data/categories_clean.csv")
offer=pd.read_csv("Data/offer_retailer_clean.csv")
brand_vectors=np.load('Data/brand_vectors.npy')
offer_vectors=np.load('Data/offer_vectors.npy')
category_vectors=np.load('Data/category_vectors.npy')
retailer_vectors=np.load('Data/retailer_vectors.npy')
offer_brand_vectors=np.load('Data/offer_brand_vectors.npy')

In [3]:
#adding in a multiplier for the brand df. This represents the proportion of receipts come from that category for that brand.
def find_brand_multiplier(b):
    df=brand[brand["BRAND"]==b]
    total_receipts=df["RECEIPTS"].sum()
    df["MULTIPLIER"]=df["RECEIPTS"]/total_receipts
    df=df[["BRAND_BELONGS_TO_CATEGORY","MULTIPLIER"]]
    return df

In [4]:
brand.head()

,MULTIPLIER,BRAND,BRAND_BELONGS_TO_CATEGORY,RECEIPTS
0,NaN,caseys gen store,tobacco products,2950931
1,NaN,caseys gen store,mature,2859240
2,NaN,equate,hair removal,893268
3,NaN,palmolive,bath body,542562
4,NaN,dawn,bath body,301844


In [5]:
cat.head()

,CATEGORY_ID,PRODUCT_CATEGORY,IS_CHILD_CATEGORY_TO
0,1f7d2fa7-a1d7-4969-aaf4-1244f232c175,red pasta sauce,pasta sauce
1,3e48a9b3-1ab2-4f2d-867d-4a30828afeab,alfredo white pasta sauce,pasta sauce
2,09f3decc-aa93-460d-936c-0ddf06b055a3,cooking baking,pantry
3,12a89b18-4c01-4048-94b2-0705e0a45f6b,packaged seafood,pantry
4,2caa015a-ca32-4456-a086-621446238783,feminine hygeine,health wellness


In [6]:
offer.head()

,OFFER,RETAILER,BRAND
0,spend 50 on a fullpriced new club membership,sams club,sams club
1,beyond meat plantbased products spend 25,NaN,beyond meat
2,good humor viennetta frozen vanilla cake,NaN,good humor
3,butterball select varieties spend 10 at dillon...,dillons food store,butterball
4,gatorade fast twitch 12ounce 12 pack at amazon...,amazon,gatorade


# 1. searches by category

In [7]:
#create a dictionary with parent categories and a list of their children categories
categories={}
parents=cat.IS_CHILD_CATEGORY_TO.unique()
for parent in parents:
    categories[parent]=list(cat[cat["IS_CHILD_CATEGORY_TO"]==parent].PRODUCT_CATEGORY.values)
categories

#create a dictionary with categories and a list of the brands that have receipts for that category.
brands={}
parents=brand.BRAND_BELONGS_TO_CATEGORY.unique()
for parent in parents:
    brands[parent]=list(brand[brand["BRAND_BELONGS_TO_CATEGORY"]==parent].BRAND.values)
brands

parents=cat.IS_CHILD_CATEGORY_TO.unique()
children=cat.PRODUCT_CATEGORY.unique()

def search_category(search):
    #create a similarity df for the search and all category name's
    cosine=cosine_similarity(model.encode([search]), category_vectors)
    sim=cat.copy(deep=True)
    sim['Cosine']=cosine.reshape(-1, 1)
    
    #Of the categories that are similar to the search, find all of the categories that also share a parent category.
    top_sim=list(sim.nlargest(1, 'Cosine')['PRODUCT_CATEGORY'].values)
    possible_cats=[]
    for c in top_sim:
        if c in parents:
            possible_cats.append(categories[c].copy())
            possible_cats.append([c])

        if c in children:
            for possible_cat in categories:
                if c in categories[possible_cat]:
                    c2=possible_cat
            possible_cats.append(categories[c2].copy())
            possible_cats.append([c2])
    possible_cats = [item for sublist in possible_cats for item in sublist] 
    #find the brands associated with these categories and weigh the brand by the number of their recipts are from these categories
    possible_brands=brand.copy(deep=True)
    possible_brands.loc[~possible_brands["BRAND_BELONGS_TO_CATEGORY"].isin(possible_cats), "MULTIPLIER"]=0
    possible_brands.loc[possible_brands["BRAND_BELONGS_TO_CATEGORY"].isin(possible_cats), "MULTIPLIER"]=1

    possible_brands=possible_brands.merge(sim,left_on="BRAND_BELONGS_TO_CATEGORY",right_on="PRODUCT_CATEGORY")
    #Then multiply the weights to get a score that also takes into account how similar the brand is to the original search
    possible_brands["cat_Score"]=(possible_brands["MULTIPLIER"]*possible_brands["Cosine"])
    possible_brands=possible_brands.groupby("BRAND").mean("cat_Score")#This score represents how likely the brand has offers related to the search term
    #find the offers from the brands above
    
    possible_offers=offer.merge(possible_brands,how="left", left_on='BRAND', right_on='BRAND')
    possible_offers=possible_offers.drop(columns=["MULTIPLIER","Cosine","RECEIPTS"])
    possible_offers['cat_Score']=possible_offers['cat_Score'].fillna(0)#if there is a brand thats in the offer df, but not the brand df, we fill the cat_score with 0 for that brand
   
    #similarity of search to offers
    cosine=cosine_similarity(model.encode([search]), offer_vectors)
    sim=offer.copy(deep=True)
    sim['Cosine']=cosine.reshape(-1, 1)
    possible_offers=possible_offers.merge(sim,how="left",on=["OFFER","RETAILER","BRAND"])
    possible_offers["Score"]=(possible_offers["cat_Score"]+possible_offers["Cosine"])/2
    possible_offers=possible_offers.drop(columns=["cat_Score","Cosine"])
    
    possible_offers=possible_offers.sort_values(by=['Score'],ascending=False)
    
    return possible_offers.reset_index(drop=True)

In [8]:
search="pasta sauce"
search_category(search).head(20)

,OFFER,RETAILER,BRAND,Score
0,barilla pesto sauce,NaN,barilla,0.475081
1,raos pasta or sauce spend 10,NaN,raos,0.430387
2,barilla pasta select varieties buy 4,NaN,barilla,0.397371
3,barilla pasta select varieties buy 2,NaN,barilla,0.394138
4,barilla pasta select varieties buy 3,NaN,barilla,0.388840
5,michael angelos sauce at walmart,walmart,michael angelos,0.247324
6,raos frozen pizza,NaN,raos,0.203387
7,red gold tomato ketchup,NaN,red gold,0.201033
8,hidden valley ranch salad dressing or secret s...,NaN,hidden valley ranch,0.193944
9,red gold tomato juice,NaN,red gold,0.189382


# 2. Searches by Brand

In [9]:
def search_brand(search):
    cosine=cosine_similarity(model.encode([search]), brand_vectors)
    
    sim=pd.DataFrame(sorted(list(set(brand["BRAND"].values))),columns=["BRAND"])
    sim['Cosine']=cosine.reshape(-1, 1)
    
    if sim.nlargest(1, 'Cosine')["Cosine"].values[0]>.7:#if there is a good match we treat the most similar brand to the search as the new search
        search=sim.nlargest(1, 'Cosine')["BRAND"].values[0]
    old_search=search#saving the old search to be used for finding the similarity between offer and search
    
    
    possible_brands=[]
    ca=[]
    for c in brands:
        if search in brands[c]:
            ca.append(c)
            possible_brands=brands[c]
    
    
    possible_brands=brand[brand["BRAND_BELONGS_TO_CATEGORY"].isin(ca)]
    possible_brands=possible_brands.merge(find_brand_multiplier(search),left_on="BRAND_BELONGS_TO_CATEGORY",right_on="BRAND_BELONGS_TO_CATEGORY")
    possible_brands["MULTIPLIER"]=(possible_brands["MULTIPLIER_x"]*possible_brands["MULTIPLIER_y"])**.5
    possible_brands=possible_brands.groupby("BRAND").sum()
    possible_offers=offer.merge(possible_brands,how='left',left_on='BRAND', right_on='BRAND')
    
    possible_offers=possible_offers.merge(sim,how='left',left_on='BRAND', right_on='BRAND')
    
    possible_offers["Brand_Score"]=(possible_offers["MULTIPLIER"]*possible_offers["Cosine"])**.5
    possible_offers.loc[possible_offers["BRAND"]==search,"Brand_Score"]=1
    
    possible_offers["Brand_Score"]=possible_offers["Brand_Score"].fillna(0)
    possible_offers=possible_offers.drop(columns=["MULTIPLIER","Cosine"])

    #similarity of search to offers
    cosine=cosine_similarity(model.encode([old_search]), offer_vectors)
    sim=offer.copy(deep=True)
    sim['Cosine']=cosine.reshape(-1, 1)
    possible_offers=possible_offers.merge(sim,on=["OFFER","RETAILER","BRAND"])
    
    possible_offers["Score"]=(possible_offers["Brand_Score"]+possible_offers["Cosine"])/2
    possible_offers=possible_offers.drop(columns=["Brand_Score","BRAND_BELONGS_TO_CATEGORY","Cosine","MULTIPLIER_x","MULTIPLIER_y","RECEIPTS"])
    
    possible_offers=possible_offers.sort_values(by=['Score'],ascending=False)
    
    return possible_offers.reset_index(drop=True)

In [10]:
search="beyond meat"
search_brand(search).head(20)

,OFFER,RETAILER,BRAND,Score
0,beyond meat plantbased products spend 15,NaN,beyond meat,0.780848
1,beyond meat plantbased products spend 20,NaN,beyond meat,0.768863
2,beyond meat plantbased products spend 25,NaN,beyond meat,0.768507
3,beyond steak plantbased seared tips 10 ounce a...,target,beyond meat,0.660037
4,beyond steak plantbased seared tips 10 ounce a...,heb,beyond meat,0.656682
5,beyond steak plantbased seared tips 10 ounce b...,heb,beyond meat,0.644833
6,beyond steak plantbased seared tips 10 ounce b...,target,beyond meat,0.640849
7,cooked perfect meatballs homestyle or turkey a...,walmart,cooked perfect,0.216957
8,cooked perfect meatballs at walmart,walmart,cooked perfect,0.204837
9,tyson products select varieties spend 15 at wa...,walmart,aidells,0.182524


# 3. Searches by Retailer

In [11]:
def search_Retailer(search):
    
    cosine=cosine_similarity(model.encode([search]), offer_vectors)
    sim=offer.copy(deep=True)
    sim['Cosine1']=cosine.reshape(-1, 1)
    
    cosine=cosine_similarity(model.encode([search]), retailer_vectors)
    sim['Cosine2']=cosine.reshape(-1, 1)
    
    cosine=cosine_similarity(model.encode([search]), offer_brand_vectors)
    sim['Cosine3']=cosine.reshape(-1, 1)
    
    possible_offers=sim
    
    possible_offers["Score"]=(possible_offers["Cosine1"]+possible_offers["Cosine2"]+possible_offers["Cosine3"])/3
    possible_offers=possible_offers.drop(columns=["Cosine1","Cosine2","Cosine3"])
    possible_offers=possible_offers.sort_values(by=['Score'],ascending=False)

    return possible_offers.reset_index(drop=True)

In [12]:
search="subway"
search_Retailer(search).head(20)

,OFFER,RETAILER,BRAND,Score
0,spend 35 at subway,subway,subway,0.875726
1,spend 15 at subway,subway,subway,0.866345
2,spend 25 at subway,subway,subway,0.865701
3,spend 10 at subway,subway,subway,0.859080
4,spend 15 at burger king,burger king,burger king,0.351341
5,spend 10 at kfc,kfc,kfc,0.337141
6,spend 40 at kfc,kfc,kfc,0.331540
7,spend 15 at kfc,kfc,kfc,0.330798
8,spend 20 at kfc,kfc,kfc,0.329632
9,spend 25 at kfc,kfc,kfc,0.322567


# Combining everything so that you can use a single search bar and not choose to search by brand, offer, or retailer

In [13]:
search="pasta sauce"

In [14]:
result = pd.concat([search_Retailer(search),search_brand(search),search_category(search)]).sort_values(by=['Score'],ascending=False)
result.drop_duplicates(subset=["OFFER","RETAILER","BRAND"],keep='first')

,OFFER,RETAILER,BRAND,Score
0,barilla pesto sauce,NaN,barilla,0.475081
1,raos pasta or sauce spend 10,NaN,raos,0.430387
2,barilla pasta select varieties buy 4,NaN,barilla,0.397371
3,barilla pasta select varieties buy 2,NaN,barilla,0.394138
4,barilla pasta select varieties buy 3,NaN,barilla,0.388840
...,...,...,...,...
372,spend 115 at tom thumb,tom thumb,tom thumb,-0.060774
373,spend 75 at randalls,randalls,randalls,-0.061101
375,spend 210 at randalls,randalls,randalls,-0.061730
374,spend 210 at randalls,randalls food markets,randalls,-0.061730
